#1.Setting Up
Run this block to install the necessary dependencies.

In [ ]:
!git clone https://github.com/indianajson/wav2lip-HD.git
basePath = "/content/wav2lip-HD"
%cd {basePath}

wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'

!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O {wav2lipPath}'/checkpoints/wav2lip_gan.pth'
#!wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O {wav2lipPath}'/checkpoints/wav2lip.pth'

!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}'/checkpoints/'

!pip install -r requirements.txt
!pip install -U librosa==0.8.1 # The process will fail without downgrading librosa
!pip install facexlib>=0.2.3
!pip install basicsr>=1.3.4.0
!pip install numpy<1.21.1
!mkdir inputs

!cd $gfpganFolderName && python setup.py develop
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P {gfpganFolderName}'/experiments/pretrained_models'

%cd {basePath}

from IPython.display import clear_output
clear_output()

print("Installation complete.")

Installation complete.


#2.Providing inputs and running the Model

In [ ]:
#@markdown Adjust the parameters below then run the code block to synthesize the speech onto your video file:

import os
outputPath = basePath+'/outputs'
inputAudio = 'output10_Pb17nq7Y.mp3' #@param{type:"string"}
inputAudioPath = basePath + '/inputs/' + inputAudio
inputVideo = 'technews-1562-ipl-final-ola-prime-plus-love-scam-motorola-edge-40-premium_UId4kWAt.mp4' #@param{type:"string"}
inputVideoPath = basePath + '/inputs/'+inputVideo
lipSyncedOutputPath = basePath + '/outputs/result.mp4'
model = "wav2lip" #@param ["wav2lip", "wav2lip_gan"] {type:"string"}


if not os.path.exists(outputPath):
  os.makedirs(outputPath)

from IPython.display import clear_output
clear_output()

!cd $wav2lipFolderName && python inference.py \
--checkpoint_path checkpoints/{model}.pth \
--face {inputVideoPath} \
--audio {inputAudioPath} \
--outfile {lipSyncedOutputPath} \
--nosmooth




#print("Video synthesis complete.")

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 300
Extracting raw audio...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-lib

#3. Boost the Resolution of the Synthesized Video using another GAN (GFPGAN)

In [ ]:
import cv2
from tqdm import tqdm
from os import path

import os

inputVideoPath = outputPath+'/result.mp4'
unProcessedFramesFolderPath = outputPath+'/frames'

if not os.path.exists(unProcessedFramesFolderPath):
  os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)


!cd $gfpganFolderName && \
  python inference_gfpgan.py -i $unProcessedFramesFolderPath -o $outputPath -v 1.3 -s 2 --only_center_face --bg_upsampler None

import os
restoredFramesPath = outputPath + '/restored_imgs/'
processedVideoOutputPath = outputPath

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np

#Get FPS of original video for writer
inputVideoPath = outputPath+'/result.mp4'
vidcap = cv2.VideoCapture(inputVideoPath)
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("The video is "+str(fps)+" FPS.")

batch = 0
batchSize = 1300
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end, end="\r")
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)
  out = cv2.VideoWriter(processedVideoOutputPath+'/output_'+str(batch).zfill(4)+'.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
  batch = batch + 1

  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

from IPython.display import clear_output
clear_output()

print("Video upscaling complete.")

Video upscaling complete.


In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips
%cd "/content/wav2lip-HD/outputs/"

video_clip = VideoFileClip("/content/wav2lip-HD/outputs/output_0000.mp4")
audio_clip = AudioFileClip(inputAudioPath)

final_clip = video_clip.set_audio(audio_clip)
final_clip.write_videofile("final_clip" + ".mp4", fps=60, audio_codec="aac")

/content/wav2lip-HD/outputs
Moviepy - Building video final_clip.mp4.
MoviePy - Writing audio in final_clipTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_clip.mp4



Moviepy - Done !
Moviepy - video ready final_clip.mp4


In [ ]:
from google.colab import files
files.download("/content/wav2lip-HD/outputs/final_clip.mp4")

#4.Running this will empty /inputs and /outputs, so you can start again, fresh.

In [ ]:
%cd /content/wav2lip-HD/

#@markdown It's recommended to always remove Outputs if you're re-running the code. You may want to preserve the input files if you are only changing one of the two inputs.

removeInputs = False #@param {type:"boolean"}
removeOutputs = True #@param {type:"boolean"}

if removeInputs == True:
  %rm inputs/*
if removeOutputs == True:
  %rm outputs/frames/*
  %rm outputs/restored_imgs/*
  %rm outputs/*

clear_output()

print("Cleared cached files.")

Cleared cached files.
